
- 기존 캐글 레포
    - 마지막말짜 9월 22일 / 8월 21일 이후의 데이터만을 사용해 학습
    - COO 매트릭스 구성
        - 8월 21일 이후의 상품, 유저를 고려하는 것이 아라
        - 전체 사용자, 전체 상품에 대해서 고려함
        
        
- 사용자는 전체 사용자로, 상품은 버리는 것으로 변경
    - 1주일 안에 계절성이 크게 변화하지는 않음
    
    
- 고객 & 아이템 세그멘트 별

In [1]:
import argparse
import logging

# import nni
# from nni.utils import merge_parameter

import implicit

from scipy.sparse import coo_matrix
from implicit.evaluation import mean_average_precision_at_k

import os
import datetime

import numpy as np
import pandas as pd
import sys
sys.path.insert(0, '../../')
from src.utils.preprocess import *
logger = logging.getLogger("nni_implicit")

In [2]:

def to_user_item_coo(df, unique_users, unique_items):
    row = df['user_id'].values
    col = df['item_id'].values    
    data = np.ones(df.shape[0])
    coo = coo_matrix((data, (row, col)), shape=(len(unique_users), len(unique_items)))
    return coo


def preprocess(trans_df, art_df, use_week_num, users2num, use_full_item):
    use_week_num += 1 # for validation 1 week
    start_date = trans_df.t_dat.max() - datetime.timedelta(7 * use_week_num)
    cur_trans_df = trans_df[trans_df['t_dat'] > start_date]

    if use_full_item == False:
        items = cur_trans_df.article_id.unique().tolist()
    else:
        items = art_df.article_id.unique().tolist()
        
    num2items = dict(list(enumerate(items)))
    items2num = dict(zip(num2items.values(), num2items.keys()))

    cur_trans_df['user_id'] = cur_trans_df.customer_id.map(users2num)
    cur_trans_df['item_id'] = cur_trans_df.article_id.map(items2num)

    train_df, val_df = split_train_valid(cur_trans_df, 0)

    coo_train = to_user_item_coo(train_df, users2num.values(), items)
    coo_val = to_user_item_coo(val_df, users2num.values(), items)
    
    csr_train = coo_train.tocsr()
    csr_val = coo_val.tocsr()
    return {'coo_train': coo_train,
            'csr_train': csr_train,
            'csr_val': csr_val
          }


In [3]:
seed = 2022
folder = '../../data/'
logger.info(os.getcwd())
art_df = pd.read_csv(os.path.join(folder, "articles.csv"))
cus_df = pd.read_csv(os.path.join(folder, "customers.csv"))
trans_df = pd.read_csv(os.path.join(folder, "transactions_train.csv"))
trans_df = make_weeknum_col(trans_df)
trans_df['t_dat'] = pd.to_datetime(trans_df['t_dat'])
users = cus_df.customer_id.unique().tolist()
num2users = dict(list(enumerate(users)))
users2num = dict(zip(num2users.values(), num2users.keys()))

In [27]:
model = 'lmf'
factors = 5
iterations = 1
regularization = 0.1
use_week_num = 2
use_full_item = True

matrices = preprocess(trans_df, art_df, use_week_num, users2num, use_full_item)    

/tmp/ipykernel_41186/764206682.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_trans_df['user_id'] = cur_trans_df.customer_id.map(users2num)
/tmp/ipykernel_41186/764206682.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_trans_df['item_id'] = cur_trans_df.article_id.map(items2num)


In [28]:
csr_train

<1371980x105542 sparse matrix of type '<class 'numpy.float64'>'
	with 462382 stored elements in Compressed Sparse Row format>

In [32]:
coo_train, csr_train, csr_val = matrices['coo_train'], matrices['csr_train'], matrices['csr_val']
if model == 'als':
    model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                iterations=iterations, 
                                                regularization=regularization, 
                                                random_state=seed)
elif model == 'bpr':
    model = implicit.bpr.BayesianPersonalizedRanking(factors=factors, 
                                                    iterations=iterations, 
                                                    regularization=regularization, 
                                                    random_state=seed)
elif model == 'lmf':                    
    model = implicit.lmf.LogisticMatrixFactorization(factors=factors, 
                                                    regularization=regularization, 
                                                    iterations=iterations, 
                                                    random_state=seed)
model.fit(csr_train, show_progress=False)

In [33]:
model

In [30]:
csr_train

<1371980x105542 sparse matrix of type '<class 'numpy.float64'>'
	with 462382 stored elements in Compressed Sparse Row format>

In [17]:
model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                iterations=iterations, 
                                                regularization=regularization, 
                                                random_state=seed)
model.fit(coo_train, show_progress=False)
map12 = mean_average_precision_at_k(model, csr_train, csr_val, K=12, show_progress=False, num_threads=4)
map12

0.0010878045365273778

In [18]:
model = implicit.bpr.BayesianPersonalizedRanking(factors=factors, 
                                                    iterations=iterations, 
                                                    regularization=regularization, 
                                                    random_state=seed)
model.fit(csr_train, show_progress=False)
map12 = mean_average_precision_at_k(model, csr_train, csr_val, K=12, show_progress=False, num_threads=4)
map12

0.0013552869156394198